In [1]:
import numpy as np 
import pandas as pd
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
# from tqdm.notebook import tqdm
from tqdm import tqdm

import albumentations as A
import albumentations.pytorch

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
train_transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1280,1280, True,1),
#    A.RandomSizedBBoxSafeCrop(384, 384),
    A.Resize(320, 320, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomResizedCrop(height=320,width=320,scale=[0.7,1.0],ratio=[0.75, 1.33],p=1.0),
#     A.RandomRain(p=0.1),
    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.99, label_fields=['labels']))

In [3]:
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
retinanet = model.resnet101(num_classes=2, device=device)
retinanet.to(device)

num_features_in of ResidualAfterFPN : 256


ResNet(
  (bn0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 24
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.81s)
creating index...
index created!


In [5]:
criterion = FocalLoss(device)
criterion = criterion.to(device)
retinanet.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-7)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=10, T_mult=2, eta_max=0.0004,  T_up=2, gamma=0.5)
# CosineAnnealingWarmRestarts

In [6]:
saved_dir = '../trained_models/resnet101_320/random_crop/'

In [7]:
# beta_list

In [8]:
EPOCH_NUM = 100
loss_per_epoch = 0.6
losses = []
cls_loss = []
box_loss = []
for epoch in range(EPOCH_NUM) :
    total_loss = 0
#     tk0 = tqdm(train_data_loader, total=len(train_data_loader), leave=False)
    EPOCH_LEARING_RATE = optimizer.param_groups[0]["lr"]

    for step, data in enumerate(tqdm(train_data_loader)) :
        if step > len(train_data_loader)/2 and epoch%10 > 0 :
            break        
#         iou_thres = np.random.uniform(low=Iou_low[epoch], high=0.5, size=None)
#         beta = np.random.uniform(low=beta_list[epoch]-0.2, high=beta_list[epoch], size=None)
#         print(step)
        images, box, label, targets = data
        batch_size = len(images)

        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)
        targets = [ t.to(device) for t in targets]

        outputs = retinanet([images, targets])
        classification, regression, anchors, annotations = (outputs)
#         print('classification', classification.shape)
#         print('regression', regression.shape)
        classification_loss, regression_loss = criterion(classification, regression, 
                                                         anchors, annotations, iou_threshold=0.3)

        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        loss = classification_loss + regression_loss 
        total_loss += loss.item()
        losses.append(loss.item())
        cls_loss.append(classification_loss.item())
        box_loss.append(regression_loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.02)
        optimizer.step()   

    print('{}th epochs loss {} lr {} '.format(epoch, total_loss/(step+1), EPOCH_LEARING_RATE ))
    if loss_per_epoch > total_loss/(step+1):
        print('best model is saved')
        torch.save(retinanet.state_dict(), saved_dir + 'best_model.pt')
        loss_per_epoch = total_loss/(step+1)
        
    scheduler.step()
#     print('epoch training time is ', time.time() - start_time)


torch.save(retinanet.state_dict(), saved_dir + 'model.pt')


  0%|          | 0/620 [00:00<?, ?it/s]

0th epochs loss 1.6922651404334652 lr 1e-07 


  0%|          | 0/620 [00:00<?, ?it/s]

1th epochs loss 1.7142349722293706 lr 0.00020005 


  0%|          | 0/620 [00:00<?, ?it/s]

...

97th epochs loss 0.22196191854965994 lr 2.382576152553092e-05 


  0%|          | 0/620 [00:00<?, ?it/s]

98th epochs loss 0.219437266055208 lr 2.260447234877746e-05 


 50%|█████     | 311/620 [04:11<04:09,  1.24it/s]


99th epochs loss 0.21711273512874657 lr 2.1389074662338725e-05 
best model is saved


In [9]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)       

loading annotations into memory...
Done (t=2.97s)
creating index...
index created!


In [ ]:
from pycocotools.cocoeval import COCOeval
import json
import torch

In [ ]:
retinanet.eval()
# start_time = time.time()
threshold = 0.5
results = []
GT_results = []
image_ids = []
cnt = 0
scores_list = []

for index, data in enumerate(tqdm(val_data_loader)) :
#     if cnt > 300 :
#         break
#     cnt += 1
    with torch.no_grad():        
        images, tbox, tlabel, targets = data
        batch_size = len(images)
#         print(tbox)
#         print(len(tbox[0]))

        c, h, w = images[0].shape
        images = torch.cat(images).view(-1, c, h, w).to(device)

        outputs = retinanet(images)
        scores, labels, boxes = (outputs)
        
        scores = scores.cpu()
        labels = labels.cpu()
        boxes  = boxes.cpu()  
        
        scores_list.append(scores)

        if boxes.shape[0] > 0:
            # change to (x, y, w, h) (MS COCO standard)
            boxes[:, 2] -= boxes[:, 0]
            boxes[:, 3] -= boxes[:, 1]
#             print(boxes)

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(boxes.shape[0]):
                score = float(scores[box_id])
                label = int(labels[box_id])
                box = boxes[box_id, :]

                # scores are sorted, so we can break
                if score < threshold:
                    break

                # append detection for each positively labeled class
                image_result = {
                    'image_id'    : dataset_val.image_ids[index],
                    'category_id' : dataset_val.label_to_coco_label(label),
                    'score'       : float(score),
                    'bbox'        : box.tolist(),
                }

                # append detection to results
                results.append(image_result)
                
        if len(tbox[0]) > 0:    

            # compute predicted labels and scores
            #for box, score, label in zip(boxes[0], scores[0], labels[0]):
            for box_id in range(len(tbox[0])):
                score = float(0.99)
                label = (tlabel[0][box_id])
                box = list(tbox[0][box_id])
                box[2] -= box[0]
                box[3] -= box[1]             

                # append detection for each positively labeled class
                image_result = {
                    'image_id'    : dataset_val.image_ids[index],
                    'category_id' : dataset_val.label_to_coco_label(label),
                    'score'       : float(score),
                    'bbox'        : list(box),
                }

                # append detection to results
                GT_results.append(image_result)                

        # append image to list of processed images
        image_ids.append(dataset_val.image_ids[index])

#        # print progress
#        print('{}/{}'.format(index, len(dataset_val)), end='\r')    

if not len(results):
    print('No object detected')
print('GT_results', len(GT_results))    
print('pred_results', len(results))    

# write output
json.dump(results, open(saved_dir + '{}_bbox_results.json'.format(dataset_val.set_name), 'w'), indent=4)
# write GT
json.dump(GT_results, open(saved_dir + '{}_GTbbox_results.json'.format(dataset_val.set_name), 'w'), indent=4)     

# print('validation time :', time.time() - start_time)



100%|██████████| 3739/3739 [05:44<00:00, 10.85it/s]


GT_results 17716
pred_results 4515


In [ ]:
# load results in COCO evaluation tool
coco_true = dataset_val.coco
coco_pred = coco_true.loadRes(saved_dir + '{}_bbox_results.json'.format(dataset_val.set_name))
coco_gt = coco_true.loadRes(saved_dir + '{}_GTbbox_results.json'.format(dataset_val.set_name))

# run COCO evaluation
# coco_eval = COCOeval(coco_true, coco_pred, 'bbox')
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids\
# coco_eval.params.catIds = [0]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   

Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.62s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.87s).
Accumulating evaluation results...
DONE (t=0.29s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.155
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.083
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.250
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.074
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.109
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

In [ ]:
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids
coco_eval.params.catIds = [0]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.74s).
Accumulating evaluation results...
DONE (t=0.23s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.194
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.300
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.241
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.500
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.146
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.214
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.214
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.174
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10

In [ ]:
coco_eval = COCOeval(coco_gt, coco_pred, 'bbox')
coco_eval.params.imgIds = image_ids
coco_eval.params.catIds = [1]
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()   

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.12s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.007
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=10